# Week 20: Execution & Market Microstructure

## 🎯 Learning Objectives

By the end of this week, you will understand:
- **Order Types**: Market, limit, stop orders
- **Market Impact**: How orders move prices
- **Execution Algorithms**: VWAP, TWAP, Implementation Shortfall
- **Optimal Execution**: Almgren-Chriss framework

---

## Why Execution Matters?

The best signal is worthless if you can't execute:
- Large orders move markets
- Slippage erodes returns
- Timing affects costs

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
print("✅ Libraries loaded!")
print("📚 Week 20: Execution & Market Microstructure")

---

## Part 1: Market Impact

### Components

1. **Temporary Impact**: Price moves during execution, reverts after
2. **Permanent Impact**: Information leakage, price stays moved

### Square-Root Law

$$\text{Impact} \approx \sigma \sqrt{\frac{Q}{V}}$$

Where:
- $\sigma$: Volatility
- $Q$: Order size
- $V$: Daily volume

In [ ]:
def market_impact(order_size, daily_volume, volatility, impact_coef=0.1):
    """Estimate market impact using square-root law"""
    participation = order_size / daily_volume
    impact = impact_coef * volatility * np.sqrt(participation)
    return impact

# Example: Trading 1M shares with 10M daily volume, 2% daily vol
order_sizes = np.array([100_000, 500_000, 1_000_000, 5_000_000])
daily_vol = 10_000_000
volatility = 0.02

print("Market Impact Estimates")
print("="*50)
print(f"Daily Volume: {daily_vol:,} shares")
print(f"Daily Volatility: {volatility:.1%}")
print(f"\n{'Order Size':<15} {'Participation':<15} {'Impact'}")
print("-"*50)

for size in order_sizes:
    impact = market_impact(size, daily_vol, volatility)
    print(f"{size:>12,} {size/daily_vol:>12.1%} {impact:>12.2%}")

In [ ]:
# Visualize market impact
participation_rates = np.linspace(0.01, 0.5, 100)
impacts = [market_impact(p * daily_vol, daily_vol, volatility) for p in participation_rates]

plt.figure(figsize=(10, 5))
plt.plot(participation_rates * 100, np.array(impacts) * 100, 'b-', linewidth=2)
plt.xlabel('Participation Rate (%)')
plt.ylabel('Market Impact (bps)')
plt.title('Market Impact vs. Participation Rate')
plt.grid(True, alpha=0.3)
plt.show()

---

## Part 2: Execution Algorithms

### TWAP - Time Weighted Average Price

Execute equal quantities at regular intervals.

### VWAP - Volume Weighted Average Price

Execute proportional to historical volume profile.

### Implementation Shortfall (IS)

Minimize cost vs. decision price.

In [ ]:
def simulate_execution(algo, total_shares, n_periods, volume_profile=None):
    """Simulate execution with different algorithms"""
    if algo == 'TWAP':
        # Equal distribution
        schedule = np.ones(n_periods) * total_shares / n_periods
    
    elif algo == 'VWAP':
        # Proportional to volume
        if volume_profile is None:
            # U-shaped intraday volume
            x = np.linspace(0, 1, n_periods)
            volume_profile = 1 + np.cos(2 * np.pi * x) * 0.5
        schedule = volume_profile / volume_profile.sum() * total_shares
    
    elif algo == 'IS_aggressive':
        # Front-loaded
        decay = np.exp(-np.linspace(0, 2, n_periods))
        schedule = decay / decay.sum() * total_shares
    
    return schedule

# Compare algorithms
total_shares = 1_000_000
n_periods = 20  # 20 time buckets

algos = ['TWAP', 'VWAP', 'IS_aggressive']
schedules = {algo: simulate_execution(algo, total_shares, n_periods) for algo in algos}

# Plot
plt.figure(figsize=(12, 4))
for algo, schedule in schedules.items():
    plt.plot(schedule, label=algo, linewidth=2)

plt.xlabel('Time Period')
plt.ylabel('Shares to Execute')
plt.title('Execution Algorithm Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---

## Part 3: Almgren-Chriss Model

### Optimal Execution

Trade-off between:
- **Market impact**: Execute slowly to reduce impact
- **Price risk**: Execute quickly to avoid adverse moves

### Objective

$$\min E[\text{Cost}] + \lambda \cdot Var[\text{Cost}]$$

Where $\lambda$ is risk aversion.

In [ ]:
def almgren_chriss_schedule(X, T, sigma, eta, gamma, risk_aversion):
    """Almgren-Chriss optimal execution schedule"""
    # Parameters
    # X: total shares to trade
    # T: trading horizon
    # sigma: volatility
    # eta: temporary impact coefficient
    # gamma: permanent impact coefficient
    # risk_aversion: lambda
    
    kappa = np.sqrt(risk_aversion * sigma**2 / eta)
    
    # Optimal trajectory
    t = np.linspace(0, T, 100)
    trajectory = X * np.sinh(kappa * (T - t)) / np.sinh(kappa * T)
    
    return t, trajectory

# Compare risk aversions
X = 1_000_000
T = 1  # 1 day
sigma = 0.02
eta = 0.0001
gamma = 0.00001

plt.figure(figsize=(10, 5))

for risk_aversion in [0.1, 1, 10]:
    t, trajectory = almgren_chriss_schedule(X, T, sigma, eta, gamma, risk_aversion)
    plt.plot(t, trajectory / X * 100, label=f'λ = {risk_aversion}', linewidth=2)

plt.xlabel('Time (days)')
plt.ylabel('Remaining Position (%)')
plt.title('Almgren-Chriss Optimal Trajectories')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Interpretation:")
print("- Low λ (risk tolerant): Trade slowly to minimize impact")
print("- High λ (risk averse): Trade quickly to reduce price risk")

---

## Part 4: Execution Cost Analysis

### Implementation Shortfall

$$IS = \text{Decision Price} - \text{Avg Execution Price}$$

### Components

- **Delay cost**: Price moved before we started
- **Market impact**: Our trading moved price
- **Timing cost**: Unfavorable execution timing

In [ ]:
def simulate_execution_cost(schedule, volatility, impact_coef):
    """Simulate execution and calculate costs"""
    n = len(schedule)
    
    # Initial price
    decision_price = 100
    price = decision_price
    
    prices = [price]
    execution_prices = []
    
    total_shares = schedule.sum()
    
    for i, shares in enumerate(schedule):
        # Random walk
        price += np.random.randn() * volatility
        
        # Market impact
        impact = impact_coef * np.sqrt(shares / total_shares)
        exec_price = price + impact
        
        prices.append(price)
        execution_prices.append(exec_price)
    
    # Calculate costs
    avg_exec_price = np.average(execution_prices, weights=schedule)
    impl_shortfall = avg_exec_price - decision_price
    impl_shortfall_bps = impl_shortfall / decision_price * 10000
    
    return impl_shortfall_bps, prices, execution_prices

# Compare algorithms
np.random.seed(42)
results = {}

print("Execution Cost Comparison (Simulated)")
print("="*50)

for algo in algos:
    costs = []
    for _ in range(1000):
        schedule = simulate_execution(algo, total_shares, n_periods)
        cost, _, _ = simulate_execution_cost(schedule, 0.5, 0.1)
        costs.append(cost)
    
    results[algo] = costs
    print(f"{algo}: Mean={np.mean(costs):.1f} bps, Std={np.std(costs):.1f} bps")

---

## Interview Questions

### Conceptual
1. What's the difference between temporary and permanent impact?
2. When would you use TWAP vs. VWAP?
3. How does risk aversion affect execution?

### Technical
1. Derive the square-root law intuitively.
2. How do you estimate impact coefficients?
3. What's the trade-off in the Almgren-Chriss model?

### Finance-Specific
1. How do you benchmark execution quality?
2. What factors affect optimal execution strategy?
3. How do you handle urgent vs. patient orders?

---

## Key Takeaways

| Algorithm | Best For | Risk |
|-----------|----------|------|
| TWAP | Predictable, simple | High if vol changes |
| VWAP | Matching benchmark | Following the crowd |
| IS/AC | Cost minimization | Model risk |